In [1]:
import sys
import numpy as np
import impl.RNN as rnn
import impl.solver as solver

In [2]:
# if __name__ == '__main__':
with open('data/text_data/japan.txt', 'r') as f:
    txt = f.read()

    X = []
    y = []

    char_to_idx = {char: i for i, char in enumerate(set(txt))}
    idx_to_char = {i: char for i, char in enumerate(set(txt))}

    X = np.array([char_to_idx[x] for x in txt])
    y = [char_to_idx[x] for x in txt[1:]]
    y.append(char_to_idx['.'])
    y = np.array(y)

In [7]:
import numpy as np
import impl.loss as loss_fun
import impl.layer as l
import impl.regularization as reg
import impl.utils as util
import impl.RNN as rnn

class GRU(rnn.RNN):

    def __init__(self, D, H, char2idx, idx2char):
        super().__init__(D, H, char2idx, idx2char)

    def forward(self, X, h_old, train=True):
        m = self.model
        Wz, Wh, Wy = m['Wz'], m['Wh'], m['Wy']
        bz, bh, by = m['bz'], m['bh'], m['by']

        X_one_hot = np.zeros(self.D)
        X_one_hot[X] = 1.
        X_one_hot = X_one_hot.reshape(1, -1)

        # concat: [h, x]
        X = np.column_stack((h_old, X_one_hot))

        # gate: h_prob
        hz, hz_cache = l.fc_forward(X, Wz, bz)
        hz, hz_sigm_cache = l.sigmoid_forward(hz)

        # signal: h_pred
        hh, hh_cache = l.fc_forward(X, Wh, bh)
        hh, hh_tanh_cache = l.tanh_forward(hh)

        # h_next and y_pred
#         h = (1. - hz) * h_old + hz * hh
        h = h_old + (hz * (hh - h_old))
#         h = h_old + (hz * hh)
        y, y_cache = l.fc_forward(h, Wy, by)

        cache = h_old, X, hz, hz_cache, hz_sigm_cache, hh, hh_cache, hh_tanh_cache, h, y_cache

        if not train:
            y = util.softmax(y)

        return y, h, cache

    def backward(self, y_pred, y_train, dh_next, cache):
        h_old, X, hz, hz_cache, hz_sigm_cache, hh, hh_cache, hh_tanh_cache, h, y_cache = cache

        dy = loss_fun.dcross_entropy(y_pred, y_train)

        dh, dWy, dby = l.fc_backward(dy, y_cache)
        dh += dh_next
        dh_old1 = (1. - hz) * dh
#         dh_old1 = dh

        dhh = hz * dh
        dhh = l.tanh_backward(dhh, hh_tanh_cache)
        dX, dWh, dbh = l.fc_backward(dhh, hh_cache)
        dh_old2 = dX[:, :self.H]

#         dhz = hh * dh - h_old * dh
        dhz = (hh - h_old) * dh
#         dhz = hh * dh
        dhz = l.sigmoid_backward(dhz, hz_sigm_cache)
        dX, dWz, dbz = l.fc_backward(dhz, hz_cache)
        dh_old3 = dX[:, :self.H]

        dh_next = dh_old1 + dh_old2 + dh_old3

        grad = dict(Wz=dWz, Wh=dWh, Wy=dWy, bz=dbz, bh=dbh, by=dby)

        return grad, dh_next

    def _init_model(self, D, C, H):
        Z = H + D

        self.model = dict(
            Wz=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wh=np.random.randn(Z, H) / np.sqrt(Z / 2.),
            Wy=np.random.randn(H, D) / np.sqrt(D / 2.),
            bz=np.zeros((1, H)),
            bh=np.zeros((1, H)),
            by=np.zeros((1, D))
        )

In [19]:
vocab_size = len(char_to_idx)

# hyper parameters
time_step = 10
# n_iter = 100000 * 2 # epochs
n_iter = 150000 # epochs
alpha = 1e-3
print_after = 1000
H = 64

In [20]:
import impl.LSTM as lstm
import impl.GRU as gru
import impl.RNN as rnn

# net = nn.LSTM(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.RNN(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
net = GRU(D=vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)
# net = nn.GRU(vocab_size, H=H, char2idx=char_to_idx, idx2char=idx_to_char)

In [21]:
solver.adam_rnn(
    net, X, y,
    alpha=alpha,
    mb_size=time_step,
    n_iter=n_iter,
    print_after=print_after
)

Iter-1000 loss: 3.3984
 djthe nwcile .ni an she espa8lom Ta tcanrwftW landene g olraS inpy icle mdedro bnt 'uamutat eh afer


Iter-2000 loss: 2.7420
o lore yhel lage thd bigJlt ny pand oSbone5 unde pusea 1ify ohec2uroll af U6elhes Cnpery wtif pbely 


Iter-3000 loss: 2.3155
r if the Ooud lamsoc, G日re(kes fon; Japan is the woc tiepero tan' on They 6 larlese mrer"apecnoacf t


Iter-4000 loss: 2.0205
uperty in the by Ad solporiand bisesise ciortolctorec Nopor heo ticontimxl cunnticed Wirgost onl"rit


Iter-5000 loss: 1.7950
cithe G8G185"-laBArof-It ind acgeres it Una, vel, cokedia o and ih the Med, culli,t al calited Eas, 


Iter-6000 loss: 1.6108
orth. The rale woslapiry streated and runec frod the sumio the Fsinex Japan inta iconury The vioper 


Iter-7000 loss: 1.4355
atand wecl. The Sil toond NiDbald of 1shto-name in the world(fore or m7er. Japlcaiser Oseath th cal 


Iter-8000 loss: 1.3003
in whe , p7opery en cand whiso pir1est milediny foutth westo cint) in the edxmaly kso id anse j

Iter-25000 loss: 0.5268
rity. Tae G8, and the 186 "ena, ahalimetoly on the Obate expalled c wrorela melitary 12shesuums meic


Iter-26000 loss: 0.5096
 mapea, Japan ty prato the Ru, fhiral of ins1Japoons in the woplocllectury meinsting purcea, of enop


Iter-27000 loss: 0.4840
higxosty Fiont ligitare in whe UNped and The wo luprcity. The forith in the Globel country in the OE


Iter-28000 loss: 0.4737
velaljof Japan of insla lome aro th the Emperor a d9. The country is the coufutoth the erprola3 Kake


Iter-29000 loss: 0.4637
ighost miorlonted and on expand ih ofo92. The wor ded sighos. The bofl of Wercod ald Senopurcen meve


Iter-30000 loss: 0.4414
f Wisto Japan was menterres in the noman Westorec. Ind revily s gheaturite eaen, the nation. Infthe 


Iter-31000 loss: 0.4305
ainal city in the world. The Glaban io of Japan") is o ald mear, ioma of Japan wes. Japan is a strat


Iter-32000 loss: 0.4126
kk, the Sigh on-lecconsed pofuranked indsaifed sictery, laef bomyounas. The world's fou

Iter-49000 loss: 0.2781
 in the Gre thed st exranded into pion a momb tea, the ward purteij, h smredgs in the Global GD Pact


Iter-50000 loss: 0.2621
stituilargest lite in 1947, Japan was purexthe 120 stand whic in the expariha mecin 18ss the forlare


Iter-51000 loss: 0.2700
e East SonoJalac citiona, the coff the Frrothimi, suriod. The fourtal ofionted wred Japan "Stoky, an


Iter-52000 loss: 0.2704
ty of Japanest: of Otetarnin Toky, hhes in the sountith and earanked first in the nea decinsh. Japan


Iter-53000 loss: 0.2429
an is ranked firstitined sing and the tat ned 98,s the worly. The four the world's third-largest exp


Iter-54000 loss: 0.2488
my by purchasing anl I perod. Ahich was in the name of the East China Seatas Kouth. Tokyo Area, whic


Iter-55000 loss: 0.2457
wo ch 1868 and the world's fourth-largest city proped Japan, the Emperor as abhing marorly tren en i


Iter-56000 loss: 0.2429
e world's largest Japan, the Eapaf with largest Japan, which is the south. The katko, h

Iter-73000 loss: 0.1946
aEmelolating particularly fr-largent ry uthe thia malies. Japanese War and World War II in the count


Iter-74000 loss: 0.2055
s. Japan Emestivl worlal cet Ind and dastoperepiopoo sivele distovimal city of Japan was proclaimed,


Iter-75000 loss: 0.2043
to expand in eaput te endenter popule en and fourth- tute iopul whinter Olyming-pargest interor lies


Iter-76000 loss: 0.1687
 The finstand is the in the north to the East China aban, mainte in the Risidenasu stantidentil the 


Iter-77000 loss: 0.1855
ed ntreedivinesd pongsto iontariess onfirs rolides ingrans whicl often a dpinter wrshR-ken us in the


Iter-78000 loss: 0.1863
an Devolof Hflome winh Senter of the Sla oph city of Tokyo, Tainly Ind end is of Hounal ComAreas the


Iter-79000 loss: 0.1704
ofith from the Empare to un a lined in 1853 when a United States flber and of Woth the Empero milita


Iter-80000 loss: 0.1785
he 1st century CE. It in the Gre the the fits restoruderestary wishu. githe prefcanter.

Iter-97000 loss: 0.1741
ke up 98.5% to tes lexom dational GDP and the warly In the world. The Greatero, in foreccalgstional 


Iter-98000 loss: 0.1713
he Glop5 iontaries, victorchy and is to c legislature called the Natice eas rits in the numbertor Ni


Iter-99000 loss: 0.1248
 of the Sla of the 12th century, which in the Country Brand Index, ranked fisl thed a unoth the 1st 


Iter-100000 loss: 0.1864
the world's largest metropolitan of soxgas to the East China Sea and Taiwheal Cowor and the core cit


Iter-101000 loss: 0.1624
ple live in the core titionam its revised constitution in 1947hislar if the north to the East China 


Iter-102000 loss: 0.1451
h the Rising duries, is the of the east ompares in erory to the West. Neard of the late 19th an peop


Iter-103000 loss: 0.1762
r and its empire of Japan's land arex. The wats a dits o period of increasing of Hwemtory to the Eas


Iter-104000 loss: 0.1568
leet pressuries insimane Emper and fourtal esing anto of Jupanes in isrch-kakkeo, 

Iter-121000 loss: 0.1338
lomeratiom. Japanesf The country is distaboturies instret. The country in Asia. Japanesed country wi


Iter-122000 loss: 0.1692
 the EIgConth loflo-lation. In ther aNdeto of Japan") is anmeral wistory. The fired agCht of the nat


Iter-123000 loss: 0.1306
udget, used forroconwith. The oppecog sh from Woped an early as the Upper end itrala led peporth lar


Iter-124000 loss: 0.1349
 four lare ian hisides, wate melion miopara in 1947, Japan has officially renounced its eftureat. Th


Iter-125000 loss: 0.1562
nhabited as end cama in the Global Peatea, inconded in 1853 whense cats the worlded Japan's name mea


Iter-126000 loss: 0.1384
pan is to the Senfude in the Gcountr Singisl bamper wrictares, is the world's largest meter and Worl


Iter-127000 loss: 0.1305
pment Index whose powho Japan is a developed country with an Empionkoded in the Fimst Japanked Asia 


Iter-128000 loss: 0.1505
 during a wot han cotso degji to early stand country in East Asia. Located Japa

Iter-145000 loss: 0.1223
ccess Hopan, the Pacida, China, Korea ofu, which insted forititionam ishing period of isolation in 1


Iter-146000 loss: 0.1280
n parth lar has refered Astan is alcor in the core city-s wital 1sing insthe to the Word's largest m


Iter-147000 loss: 0.1358
–2016 and its empited Japan to an a decins ranked and the world's third-largest exporter and fourth-


Iter-148000 loss: 0.1209
land 1868, Japan is a st in CEm Niprover was the Upprocoun, the nasiof limperomonsof-useates of the 


Iter-149000 loss: 0.1198
 a is the highest milliont an suust. The Grory. Abothan wres hesth largest are Honshu, Hokeke, shufr


Iter-150000 loss: 0.1197
's history. From Ware war, st ricity of Japan") is an islands in the Ochas of sixtht Sea of 1shict u




In [ ]:
=========================================================================
Iter-148000 loss: 0.1209
=========================================================================
land 1868, Japan is a st in CEm Niprover was the Upprocoun, the nasiof limperomonsof-useates of the 
=========================================================================


=========================================================================
Iter-149000 loss: 0.1198
=========================================================================
 a is the highest milliont an suust. The Grory. Abothan wres hesth largest are Honshu, Hokeke, shufr
=========================================================================


=========================================================================
Iter-150000 loss: 0.1197
=========================================================================
's history. From Ware war, st ricity of Japan") is an islands in the Ochas of sixtht Sea of 1shict u
=========================================================================


Out[21]:
<__main__.GRU at 0x10d0830f0>

In [ ]:
=========================================================================
Iter-99000 loss: 0.1560
=========================================================================
 of the UN, the G7, the G8, and the world's fourth-largest economy, wites ecatroped bote period. Tov
=========================================================================


=========================================================================
Iter-100000 loss: 0.1881
=========================================================================
the worth-leven periotan and in 1945 following the atom inter and Resilled the "Land of the Rssicand
=========================================================================


Out[13]:
<__main__.GRU at 0x10d084470>

In [ ]:
=========================================================================
Iter-297000 loss: 0.1208
=========================================================================
ry with a Unised Japan's history. From the 12th century, which isHomAreat Divi al China, folicter Ol
=========================================================================


=========================================================================
Iter-298000 loss: 0.0927
=========================================================================
 the number of Nobel laureates of anI arean Index. Japan was the first cores. Japan is a developed c
=========================================================================


=========================================================================
Iter-299000 loss: 0.1143
=========================================================================
ake up Jcaly Id in 1868 and the Empire of Japan was proclaimed, ater popJdallky, thicalic and eanly 
=========================================================================


=========================================================================
Iter-300000 loss: 0.1084
=========================================================================
uled by suchaed into a and in 1945 follodud of Hurigh in the world. The Greater Tokyo Area, which in
=========================================================================


Out[27]:
<__main__.GRU at 0x10d06ed68>

In [ ]:

=========================================================================
Iter-198000 loss: 0.1331
=========================================================================
tary shoguns who ruled in the nainty n mainterd Taiwat lors latulaty und Sunarend thia dudg oonaly r
=========================================================================


=========================================================================
Iter-199000 loss: 0.1386
=========================================================================
th largest imf in the korlation mainlatI un the eipan endand of the Rising Sun". Japan is a stratovo
=========================================================================


=========================================================================
Iter-200000 loss: 0.0987
=========================================================================
hest-ranked Asian countrord with the Since caly towertor Jaty in the country in East Asia. Located i
=========================================================================


Out[11]:
<__main__.GRU at 0x10d053f98>

In [ ]:
=========================================================================
Iter-99000 loss: 0.1890
=========================================================================
 of No20th bex, mfen eped consic th led the fourth leading globan in the name of the Emperor. Japan 
=========================================================================


=========================================================================
Iter-100000 loss: 0.1673
=========================================================================
the early 1shthicaped and the forchest urerlate 19th and the world's fourth-largest economy by purch
=========================================================================


Out[24]:
<__main__.GRU at 0x10d1f7b00>

In [ ]:
=========================================================================
Iter-100000 loss: 0.3913
=========================================================================
the 1pman a inusy rotory surrowho tOkike upporter wor peven des The easty porterized itivecend perce
=========================================================================


Out[7]:
<hipsternet.neuralnet.RNN at 0x7fc2f5182e80>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1615
=========================================================================
the name of the 12th century until 1868, Japan was ruled by successime Wer of 1941, which country is
=========================================================================


Out[16]:
<hipsternet.neuralnet.LSTM at 0x7f67fe0c9978>

In [ ]:
=========================================================================
Iter-100000 loss: 0.1333
=========================================================================
the Coun resilate a in the population. Russided in the OECD and the world's fourth-largest econompic
=========================================================================


Out[10]:
<hipsternet.neuralnet.GRU at 0x7fc2cec1e1d0>